In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from dash import ctx

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# Sort by rec_num
df = df.sort_values(by='rec_num', ascending=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date

app.layout = html.Div([
    #    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Rescue Austin'))),
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={"height": "120px", "display": "block", "margin": "0 auto"}),
    html.H2("CS-340 Dashboard | Jordan Caizza", style={"textAlign": "center"}),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id="rescue_filter",
            options=[
                {"label": "Water Rescue", "value": "water"},
                {"label": "Mountain Rescue", "value": "mountain"},
                {"label": "Disaster Rescue", "value": "disaster"},
                {"label": "Reset", "value": "reset"},
            ],
            value="reset",
            inline=True,
            style={"margin": "6px 0"}
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size=10,
                         sort_action='native',
                         filter_action='native',
                         sort_mode='multi',
                         row_selectable='single',
                         selected_rows=[0],
                         style_table={'overflowX': 'auto'},
                         style_cell={
                             'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                             'overflow': 'hidden',
                             'textOverflow': 'ellipsis',
                         }
                        ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',

                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output("datatable-id", "data"),
               Output("datatable-id", "selected_rows"),
               Output("datatable-id", "page_current")],
              [Input("rescue_filter", "value")],
              prevent_initial_call=False)
def update_dashboard(filter_type):
    docs = shelter.find_by_rescue_type(filter_type)
    dff = pd.DataFrame.from_records(docs)
    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)
    if "rec_num" in dff.columns:
        dff = dff.sort_values(by="rec_num", ascending=True)

    # Always select the first row and go to page 0 after a filter change
    selected = [0] if len(dff) else []
    return dff.to_dict("records"), selected, 0

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Convert the visible (filtered) table data to a DataFrame
    dff = pd.DataFrame(viewData or [])
    if dff.empty or "breed" not in dff.columns:
        return []  # nothing to draw yet

    # Count breeds and take the top 10 for clarity
    counts = (
        dff["breed"]
        .dropna()
        .value_counts()
        .reset_index()
        .rename(columns={"index": "breed", "breed": "count"})
        .head(10)
    )

    fig = px.pie(
        counts,
        names="breed",
        values="count",
        title="Top Breeds by Quantity"
    )
    fig.update_traces(textinfo="percent+label")

    return [dcc.Graph(figure=fig)]

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def highlight_row(selected_rows):
    if not selected_rows:
        return [{'if': {'row_index': 0}, 'background_color': '#D2F3FF'}]
    return [{'if': {'row_index': i}, 'background_color': '#D2F3FF'} 
            for i in selected_rows]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData or [])
    row = 0 if not index else index[0]
    try:
        lat = float(dff.iloc[row, 13])
        lon = float(dff.iloc[row, 14])
        breed = dff.iloc[row, 4]
        name = dff.iloc[row, 9]
    except Exception:
        return []  # nothing to render

    return [dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H1("Animal Name"), html.P(name)])
            ])
        ]
    )]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:10397/
